In [1]:
from models.trainModel import load_model
from utils.dataset import load_dataset
from utils.common import remove_digit_from_labels, one_hot_to_number
from PIL import Image


def get_images(teaching_set, ai):
    images = []
    X,Y,labels = load_dataset()  
    found = []
    title = []
    for example in teaching_set:
        example_label, value = example
        found = False
        for i in range(len(X)-1,0,-1):
            corr_label = "".join(remove_digit_from_labels(labels[i]))
            
            if corr_label == example_label:
                if one_hot_to_number(ai.predict(X[i]))== value:
                    images.append(Image.fromarray(X[i]))
                    title.append(value)
                    found= True
                    break
        if not found:
            print("Could not find a data instance with the combination of: ", example_label, value)
    for img,title in zip(images,title):
        img.show(title=title)
    return images

In [2]:
from utils.common import get_all_letter_combinations
from random import randint, shuffle,choice
from PIL import Image
from utils.common import one_hot_to_number
from TA.delta.sumOfExamples import SumOfExamples as delta


def generate_random_teaching_set_of_size_k(k,ai, e=1):
    X,Y,labels = load_dataset()  
    random_combo = list(zip(X,labels))
    shuffle(random_combo)
    
    img_to_show = []
    ai_values = []
    letters = []

    while len(letters) < k:
        x,l = choice(random_combo)
        l = "".join(remove_digit_from_labels(l))
        if l not in letters:
            letters.append(l)
            img_to_show.append(Image.fromarray(x))
            ai_values.append(one_hot_to_number(ai.predict(x)))
                
    for i in range(len(img_to_show)):
        print(letters[i], ai_values[i])
        img_to_show[i].show()

# Get random teaching set
Select random unique letter groups, s.t. delta(ts_s) = delta(ts_r) +- e.
e = 1

In [3]:
from utils.common import get_all_letter_combinations
from random import randint, shuffle,choice
from TA.delta.sumOfExamples import SumOfExamples


def get_random_teaching_set(delta_score, epsilon,ai):
    delta = SumOfExamples()
    teaching_set = []
    while True:
        teaching_set = []
        pos_letter = get_all_letter_combinations()
        one_true = False
        one_false = False
        while delta.get_complexity_of_subset(teaching_set) < delta_score - epsilon:
            lg = choice(pos_letter)
            teaching_set.append(lg)
            pos_letter.remove(lg)
            if get_most_prop_val_given_lg(ai,lg) == 1:
                one_true = True

            else:
                one_false = True
        if delta.get_complexity_of_subset(teaching_set) <= delta_score + epsilon:
            if one_true and one_false:
                break
    return sorted(teaching_set)
    

# Given random letters, find random images

In [4]:
from random import choice
from utils.dataset import load_dataset
from PIL import Image


def get_images(letter_groups , ai):
    X,Y,labels = load_dataset()  
    data_labels = list(zip(X,labels))
    title = []
    images = []
    for lg in letter_groups:
        found = False
        while not found:
            (curr_x,label) = choice(data_labels)
            label = "".join(remove_digit_from_labels(label))
            if label == lg:
                images.append(curr_x)
                title.append(lg)
                found = True

    return images, title

# Find epsilon of given teaching set

In [5]:
from TA.delta.sumOfExamples import SumOfExamples
def get_delta_score(teaching_set):
    return SumOfExamples().get_complexity_of_subset(teaching_set=teaching_set)

def get_lg_from_res(data):
    return data.split(",")[0].split("-")

In [6]:

get_delta_score(get_lg_from_res("A-AB-ABCD-ABD-AC-ACD-AD-B-BC-BCD,1-0-0-1-0-1-0-1-0-1"))

23

In [7]:
from utils.dataset import load_dataset
from utils.common import remove_digit_from_labels
from TA.Lambda.mean_square_error import MSE
from models.trainModel import load_model
import random

def prop_map_display(model):
    print("loading...")
    X, y, labels = load_dataset()
    print("loading done!")
    labels = remove_digit_from_labels(labels)
    mse = MSE()
    prop_map = mse._get_probaility_map_ai(ai_model=model, valid_X=X, valid_labels=labels)
    for elem in prop_map.keys():
        print(elem, str(prop_map[elem]*100).replace(".",","))
        
def get_most_prop_val_given_lg(model,lg):
    X, y, labels = load_dataset()
    labels = remove_digit_from_labels(labels)
    mse = MSE()
    prop_map = mse._get_probaility_map_ai(ai_model=model, valid_X=X, valid_labels=labels)
    return round(prop_map[lg])

In [26]:
from models.trainModel import load_model
ai = load_model(f"Phi_v2_5_nD")

from utils.common import one_hot_to_number
curr_ts = get_lg_from_res("-D,1-0")
delta = get_delta_score(curr_ts)
print(delta)
r_TS = get_random_teaching_set(delta,1,ai)
print(get_delta_score(r_TS))

print(r_TS)
#images, titles = get_images(r_TS, ai)
ts_str = "-".join(r_TS) + "," + "-".join(map(lambda x: str(get_most_prop_val_given_lg(ai,x)), r_TS))
print(ts_str)
#for image, title in zip(images, titles):
#    print(title,one_hot_to_number(ai.predict(image)))
    #Image.fromarray(image).show()

1.1
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map...
loading probability map.

In [9]:
for image, title in zip(images, titles):
    print(title,one_hot_to_number(ai.predict(image)))

NameError: name 'images' is not defined

In [ ]:
import random
for i in range(16):
    print(random.randint(0,1), end=" ")
print()
    

In [ ]:
prop_map_display(ai)
get_most_prop_val_given_lg(ai, "C")